In [1]:
def cross(a, b):
    return [s+t for s in a for t in b]

In [45]:
rows = 'ABCDEFGHI'
cols = '123456789'
grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
grid3 = '8...........26..5...98....47.3....2.....4..815....7.3..5...924....7.....91.......'
grid4 = '..7.6..31...3.16.2....8...91.6..3.....5.........9...5737........29..........34...'
grid5 = '.46..78....89..7..5...2........62..44.......7....5..1...2.......15....2......8.6.'
grid6 = '7.8...5.2.2..1..8....8.......3.....9.5..7..4.6.....3.......6....4..5..7.5.9...6.3'
grid7 = '..7..8..4.1..4..5.4..6..3..1..4..8...7..1..9...3..2..5..6..4..9.3..9..8.8..2..6..'

boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#self defined
history = []


In [3]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [4]:
def grid_values(grid):
    
    grid_dict = {}
    dpossibleVals = '123456789'
    
    row_units = [cross(r, cols) for r in rows]
    
    for r in range(len(row_units)):
        for c in range(len(row_units[1])):
            if grid[r*9+c] != '.':
                grid_dict[row_units[r][c]] = grid[r*9+c]
            else:
                grid_dict[row_units[r][c]] = dpossibleVals
    
    return grid_dict

In [5]:
def eliminate(values):
    for key, val in values.items():
        if len(val) == 1:
            for u in range(3):
                for b in units[key][u]:
                
                    if val in values[b] and val != values[b]:
                        values[b] = values[b].replace(val, '')
                        #values, history = assignValue(history, values, 
                                        #b, values[b].replace(val, ''), "eliminate")
    return values

In [6]:
def only_choice(values):
    
    """
    for units in (row_units, column_units, square_units):
        for sq in units:
            for d in range(len(cols)):
                current_d_count = 0
                current_digit = d+1
            
                for u in sq:
                    if str(current_digit) in str(values[u]) and str(current_digit) != values[u]:
                        current_d_count += 1
                        last_unit = u
                if current_d_count == 1:
                    values[last_unit] = str(current_digit)
    """ 

    
    for box, nhb_units_array in units.items():

        possible_values = values[box]
        #print(box)
        
        if len(possible_values) > 1: # not consider those assigned
            for poss_val_index in range(len(possible_values)):
                
                this_val = possible_values[poss_val_index]
                
                
                
                for nhb_units_row in nhb_units_array:  #nhb_units_array has len of 3
                    
                    count = 0
                    
                    for u in nhb_units_row:
                        #print(u)
                        
                        if this_val in values[u] and box != u:
                            count += 1
                            
                    if count == 0:
                        #print(box, this_val)
                        values[box] = this_val
                        
                        #values, history = assignValue(history, values, 
                                        #box, this_val, "only_choice")
                        
    return values


In [7]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)
        
        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)
        
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [8]:
def isSolved(values):
    
    solvedStatus = True
    
    for unit in (column_units, row_units, square_units):
        for arr in unit:
            sub_total = 0
            for u in arr:
                sub_total += int(values[u])
            
            if sub_total != 45:
                solvedStatus = False
    
    return solvedStatus

In [9]:
def assignValue1(history, values, box, new_val, funcName):
    
    transaction = {}
    transaction['box'] = box
    transaction['changefrom'] = values[box]
    transaction['changechangeto'] = new_val
    transaction['functionName'] = funcName
    
    values[box] = new_val
    
    history.append(transaction)
    
    return values, history

In [66]:
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    
    # Choose one of the unfilled squares with the fewest possibilities
    
    #####
    if type(values) != bool:
        solvedStatus = True
        #print(str(type(values)) + str(3))
        for unit in (column_units, row_units, square_units):
            for arr in unit:
                sub_total = 0
                for u in arr:
                    sub_total += int(values[u])

                if sub_total != 45:
                    solvedStatus = False


        status = solvedStatus
    else:
        status = False
    #####
    
    if status == False: #trial and error approach
        
        # find the box with fewest possible options
        min_box = ''
        min_length = 9
        for r in row_units:
            for u in r:
                if type(values) != bool and len(values[u]) > 1 and len(values[u]) < min_length:
                    min_box = u
                    min_length = len(values[u])
        
        
        #display(values)
        if min_box != '': # try different values and continue down the tree
            for pos_val in values[min_box]:
                    
                tmp_values = dict(values)
                    
                #tmp_values, history = assignValue(history, tmp_values, min_box, pos_val, "search")
                #print(min_box, tmp_values[min_box], pos_val)
                #if min_box == 'B3':
                #display(tmp_values)
                #print("")
                
                tmp_values[min_box] = pos_val
                

                #tmp_values, status = search(tmp_values)
                tmp_val = search(tmp_values)
                #print(type(tmp_val))
                if type(tmp_val) != bool:
                #and tmp_val != False:
                    #print(tmp_val)
                    values = tmp_val
                    return values
        else:
            #all boxes filled with single value but wrong solution!
            
            #####
            if type(values) != bool:
                solvedStatus = True
                #print(str(type(values)) + str(3))
                for unit in (column_units, row_units, square_units):
                    for arr in unit:
                        sub_total = 0
                        for u in arr:
                            sub_total += int(values[u])

                        if sub_total != 45:
                            solvedStatus = False


                status = solvedStatus
            else:
                status = False
            #####
            
            if status == False:
                values = status

            return values

    else:
        #print("puzzle solved")
        #display(values)
        
        if status == False:
            values = status
        
        return values
            
    
    # Now use recursion to solve each one of the resulting sudokus, 
    # and if one returns a value (not False), return that answer!

    # If you're stuck, see the solution.py tab!
    
    #####
    if type(values) != bool:
        solvedStatus = True
        #print(str(type(values)) + str(3))
        for unit in (column_units, row_units, square_units):
            for arr in unit:
                sub_total = 0
                for u in arr:
                    sub_total += int(values[u])

                if sub_total != 45:
                    solvedStatus = False


        status = solvedStatus
    else:
        status = False
    #####
    
    if status == False:
        values = status
    
    return values

In [67]:
grid_dict1 = grid_values(grid7)
#display(grid_dict2)
grid_dict2 = search(grid_dict1)
#print(grid_dict2)
if type(grid_dict2) != bool:
    print(isSolved1(grid_dict2))
    display(grid_dict2)

True
3 5 7 |9 2 8 |1 6 4 
6 1 2 |3 4 7 |9 5 8 
4 8 9 |6 5 1 |3 7 2 
------+------+------
1 6 5 |4 7 9 |8 2 3 
2 7 8 |5 1 3 |4 9 6 
9 4 3 |8 6 2 |7 1 5 
------+------+------
7 2 6 |1 8 4 |5 3 9 
5 3 4 |7 9 6 |2 8 1 
8 9 1 |2 3 5 |6 4 7 
